# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [58]:
#Your code here
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

treated = len(df[df['group'] == 'experiment'])
control = len(df[df['group'] == 'control'])
total = treated + control
print('Control: ', control, '\nTreated: ', treated, '\nTotal: ', total)


Control:  4264 
Treated:  3924 
Total:  8188


In [59]:
df['id'].value_counts()
'''There are some viewers that have more than 1 entry. Let's check how many they are'''
df.duplicated('id').value_counts()

'''There are 1860 id duplicates. Let's check some of them'''

df.sort_values('id')
'''It seems that a entry might be doubled corresponding to a given action that was undertaken.'''
df_dup = df[df.duplicated('id', False)]

df_dup['action'].value_counts()
'''So 1,860 users clicked and viewed content out of an overall number of 8,188'''

'So 1,860 users clicked and viewed content out of an overall number of 8,188'

In [42]:
df_ndup = df[~df.duplicated('id', False)]
df_ndup['id'].value_counts()
'''We have 4,468 unique id values. Let's check how many of those only clicked'''

df_ndup['action'].value_counts()

'''the non-duplicated values only show views without clicks. So we end up with 4 groups: 
treated that clicked and viewed, treated that just clicked, control that clicked and view and control that just clicked''' 

'''I would then suggest checking whether the differences between control and treated that clicked and viewed are
statistically significant. To do so, we will need to embed the overall number of individuals (without double counting)
and check the difference between these two groups'''


'I would then suggest checking whether the differences between control and treated that clicked and viewed are\nstatistically significant. To do so, we will need to embed the overall number of individuals (without double counting)\nand check the difference between these two groups'

In [43]:
df_ndup['group'].value_counts()

control       2400
experiment    2068
Name: group, dtype: int64

In [44]:
df_dup['group'].value_counts()

control       1864
experiment    1856
Name: group, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [70]:
df_group = df.groupby(['group','action']).count()

In [71]:
df_group

timestamp    id  Click
group      action                        
control    click         932   932    932
           view         3332  3332      0
experiment click         928   928    928
           view         2996  2996      0

In [81]:
df['count'] = 1
df.head()

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()


0.2797118847539016 0.3097463284379172
-0.0976504334143771
0.5388938669081622


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [84]:
#Control Rate
dfc = 932 / 3332
var = treated * dfc * (1 - dfc)
std = np.sqrt(var)
print(std)

28.96489414366718


In [85]:
df['count'] = 1
df.head()

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [89]:
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [91]:
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [92]:
#Your code here
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

3.6625360854823588


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
We still reject the null. The website appears to be succesful in its approach.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.